In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# Conextions with Chrome Driver
ser = Service("./chromedriver")
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

# Definition of drivers
driver_urls = webdriver.Chrome(options=options)
driver_last_page = webdriver.Chrome(options=options)
driver_products = webdriver.Chrome(options=options)

In [10]:
def get_max_page_catalog():
    # Getting the number of the last page into the catalog
    last = 10000000
    page = f'https://www.farmaciasahumada.cl/medicamentos.html?p={last}'
    driver_last_page.get(page)
    last_page = driver_last_page.find_elements(By.XPATH,'//*[@id="maincontent"]/div[4]/div[1]/div[4]/div[2]/ul/li[1]/a')
    for value in last_page:
            url_last_page = (value.get_attribute('href'))
            n = url_last_page[-3:]
            n = int(n)
    return n

In [12]:
get_max_page_catalog()

222

In [13]:
def get_url_from_catalog_ahumada(n):
    list_url = []
    for i in range(n):
        page = f'https://www.farmaciasahumada.cl/medicamentos.html?p={i+1}'
        driver_urls.get(page)
        list = driver_urls.find_elements(By.XPATH,"//strong[contains(@class,'product name product-item-name truncate')]/a[contains(@class,'product-item-link')]")
        for i in range(len(list)):
                    url = list[i].get_attribute('href')
                    list_url.append(url)
        time.sleep(2)
    return list_url

In [14]:
get_url_from_catalog_ahumada(2)

['https://www.farmaciasahumada.cl/prestat-75-mg-x-40-comprimidos.html',
 'https://www.farmaciasahumada.cl/kitadol-500-mg-x-24-comprimidos.html',
 'https://www.farmaciasahumada.cl/prestat-150-mg-x-40-comprimidos.html',
 'https://www.farmaciasahumada.cl/glicenex-sr-1000-mg-x-30-comprimidos-de-liberacion-prolongada.html',
 'https://www.farmaciasahumada.cl/pedialyte-60-manzana-x-500-ml-solucion-oral.html',
 'https://www.farmaciasahumada.cl/shot-c-vitamina-c-y-zinc-x-30-capsulas.html',
 'https://www.farmaciasahumada.cl/toptear-0-4-x-10-ml-solucion-oftalmica.html',
 'https://www.farmaciasahumada.cl/fibrasol-naranja-68-x-200-g-polvo-para-suspension-oral.html',
 'https://www.farmaciasahumada.cl/prestat-50-mg-40-comp.html',
 'https://www.farmaciasahumada.cl/pedialyte-45-frutilla-x-500-ml-solucion-oral.html',
 'https://www.farmaciasahumada.cl/kitadol-1-g-x-18-comprimidos.html',
 'https://www.farmaciasahumada.cl/canesten-1-x-20-g-crema-topica.html',
 'https://www.farmaciasahumada.cl/aloelax-x-50-

In [17]:
def extract_info_web_site_ahumada(list_url):
    list_product_variables = []
    for url in list_url:
        dic_product_variables = {}
        driver_products.get(url)
        # URL
        dic_product_variables["link"] = url
        # Laboratory
        try:
            dic_product_variables["lab"] = driver_products.find_element(By.XPATH,"// h3[contains(@class,'product-brand')]").text
        except NoSuchElementException:
            dic_product_variables["lab"] = 'No Laboratory'
        # Description
        try:
            dic_product_variables["Product_description"] = driver_products.find_element(By.XPATH,"// div[contains(@class,'page-title-wrapper product')]/h1[contains(@class,'page-title')]/span[contains(@class,'base')]").text
        except NoSuchElementException:
            dic_product_variables["Product_description"] = 'No description'
        # Special price
        try:
            dic_product_variables["Special_price"] = driver_products.find_element(By.XPATH,"//span[contains(@class,'special-price')]/span[contains(@class,'price-container price-final_price tax weee prueba')]/span[contains(@class,'price')]").text
        except NoSuchElementException:
            dic_product_variables["Special_price"] = 'No special price'
        # normal price
        try:
            dic_product_variables["Normal_price"] = driver_products.find_element(By.XPATH,"//span[contains(@class,'old-price')]/span[contains(@class,'price-container price-final_price tax weee')]/span[contains(@class,'price-wrapper ')]/span[contains(@class,'price')]").text
        except NoSuchElementException:
            dic_product_variables["Normal_price"] = 'No normal price'
        # Internet Price
        try:
            dic_product_variables["Internet_price"] = driver_products.find_element(By.XPATH,"//span[contains(@class,'price-wrapper')]/span[contains(@class,'price')]").text
        except NoSuchElementException:
            dic_product_variables["Internet_price"] = 'No internet price'
        # SKU
        try:
            dic_product_variables["sku"] = driver_products.find_element(By.XPATH,"//*[@id='maincontent']/div[3]/div/div[2]/div[3]/div[2]/div/div").text
        except NoSuchElementException:
            dic_product_variables["sku"] = 'No sku'

        list_product_variables.append(dic_product_variables)

    df_ahumada = pd.DataFrame(list_product_variables)
    return df_ahumada


In [18]:
extract_info_web_site_ahumada(get_url_from_catalog_ahumada(2))

,link,lab,Product_description,Special_price,Normal_price,Internet_price,sku
0,https://www.farmaciasahumada.cl/prestat-75-mg-...,Bago,Prestat 75 mg x 40 Comprimidos,No special price,No normal price,$12.295,75541
1,https://www.farmaciasahumada.cl/kitadol-500-mg...,Chile,Kitadol 500 mg x 24 Comprimidos,No special price,No normal price,$1.395,15414
2,https://www.farmaciasahumada.cl/prestat-150-mg...,Bago,Prestat 150 mg x 40 Comprimidos,No special price,No normal price,$23.495,75427
3,https://www.farmaciasahumada.cl/glicenex-sr-10...,Bago,Glicenex SR 1000 mg x 30 Comprimidos de Libera...,No special price,No normal price,$12.695,87201
4,https://www.farmaciasahumada.cl/pedialyte-60-m...,Abbott,Pedialyte 60 Manzana x 500 mL Solución Oral,No special price,No normal price,$4.395,5866
5,https://www.farmaciasahumada.cl/shot-c-vitamin...,Shot C,Shot C Vitamina C y Zinc x 30 Capsulas,No special price,No normal price,$6.995,88680
6,https://www.farmaciasahumada.cl/toptear-0-4-x-...,Toptear,"Toptear 0,4 % x 10 mL Solución Oftálmica",No special price,No normal price,$14.795,49627
7,https://www.farmaciasahumada.cl/fibrasol-naran...,Maver S.A.,Fibrasol Naranja 68 % x 200 g Polvo Para Suspe...,No special price,No normal price,$16.295,75523
8,https://www.farmaciasahumada.cl/prestat-50-mg-...,Bago,PRESTAT 50 MG. 40 COMP.,No special price,No normal price,$10.695,82703
9,https://www.farmaciasahumada.cl/pedialyte-45-f...,Abbott,Pedialyte 45 Frutilla x 500 mL Solución Oral,No special price,No normal price,$4.395,74371


In [6]:
# CREATE TABLE Catalog_cruz_verde
def create_if_not_exist_ahumada():
    c.execute("""CREATE TABLE IF NOT EXISTS Catalog_Ahumada(link TEXT, 
                                            lab TEXT, 
                                            Product_description TEXT, 
                                            Special_price TEXT,
                                            Normal_price TEXT,
                                            sku TEXT)""")

In [7]:
# INSERT DATA INTO SQL TABLE
def insert_df_sql_ahumada(df_ahumada):
    for ind in df_ahumada.index:
        c.execute("insert into Catalog_Ahumada VALUES(?,?,?,?,?,?)",\
        (df_ahumada.link[ind],\
        df_ahumada.lab[ind],\
        df_ahumada.Product_description[ind],\
        df_ahumada.Special_price[ind],\
        df_ahumada.Normal_price[ind],\
        df_ahumada.sku[ind]))

In [14]:
conn = sqlite3.connect('Catalog_pharmacies.db')
c = conn.cursor()

create_if_not_exist_ahumada()
conn.commit()

n = get_max_page_catalog()
list_url= get_url_from_catalog_ahumada(n)
df_ahumada = extract_info_web_site_ahumada(list_url)
insert_df_sql_ahumada(df_ahumada)
conn.commit()
c.close()
conn.close()

In [20]:

url = 'https://www.farmaciasahumada.cl/kopodex-1000-mg-x-30-comprimidos-recubiertos.html'

list_product_variables = []

dic_product_variables = {}
driver_products.get(url)
# URL
dic_product_variables["link"] = url
# Laboratory
try:
    dic_product_variables["lab"] = driver_products.find_element(By.XPATH,"// h3[contains(@class,'product-brand')]").text
except NoSuchElementException:
    dic_product_variables["lab"] = 'No Laboratory'
# Description
try:
    dic_product_variables["Product_description"] = driver_products.find_element(By.XPATH,"// div[contains(@class,'page-title-wrapper product')]/h1[contains(@class,'page-title')]/span[contains(@class,'base')]").text
except NoSuchElementException:
    dic_product_variables["Product_description"] = 'No description'
# Special price
try:
    dic_product_variables["Special_price"] = driver_products.find_element(By.XPATH,"//span[contains(@class,'special-price')]/span[contains(@class,'price-container price-final_price tax weee prueba')]/span[contains(@class,'price')]").text
except NoSuchElementException:
    dic_product_variables["Special_price"] = 'No special price'
# normal price
try:
    dic_product_variables["Normal_price"] = driver_products.find_element(By.XPATH,"//span[contains(@class,'old-price')]/span[contains(@class,'price-container price-final_price tax weee')]/span[contains(@class,'price-wrapper ')]/span[contains(@class,'price')]").text
except NoSuchElementException:
    dic_product_variables["Normal_price"] = 'No normal price'
# Internet Price
try:
    dic_product_variables["Internet_price"] = driver_products.find_element(By.XPATH,"//span[contains(@class,'price-wrapper')]/span[contains(@class,'price')]").text
except NoSuchElementException:
    dic_product_variables["Internet_price"] = 'No internet price'
# SKU
try:
    dic_product_variables["sku"] = driver_products.find_element(By.XPATH,"//*[@id='maincontent']/div[3]/div/div[2]/div[3]/div[2]/div/div").text
except NoSuchElementException:
    dic_product_variables["sku"] = 'No sku'

list_product_variables.append(dic_product_variables)
list_product_variables

[{'link': 'https://www.farmaciasahumada.cl/kopodex-1000-mg-x-30-comprimidos-recubiertos.html',
  'lab': 'Recalcine S.A.',
  'Product_description': 'Kopodex 1000 mg x 30 Comprimidos Recubiertos',
  'Special_price': 'No special price',
  'Normal_price': 'No normal price',
  'Internet_price': '$45.295',
  'sku': '46545'}]

In [8]:
list_product_variables

[{'link': 'https://www.farmaciasahumada.cl/prestat-75-mg-x-40-comprimidos.html',
  'lab': 'Bago',
  'Product_description': 'Prestat 75 mg x 40 Comprimidos',
  'Special_price': 'No special price',
  'Normal_price': 'No normal price',
  'sku': '75541'}]